# Inference

In [11]:
import numpy as np
import pandas as pd
import sklearn
import joblib

In [7]:
#Load from file
print('Loading features...')
dataset_test = pd.read_csv('features_test.csv')
num_features = 768
features_test = dataset_test.iloc[:,0:num_features].to_numpy()
target_test = np.zeros((features_test.shape[0],3))
target_test[:,0] = dataset_test['x'] 
target_test[:,1] = dataset_test['y']
target_test[:,2] = dataset_test['z']
print(features_test.shape)
print(target_test.shape)
print('Done!')

Loading features...
(3974, 768)
(3974, 3)
Done!


In [13]:
#restoring the classifier model from file
joblib_file = 'doa_model.pkl'
restored_model = joblib.load(joblib_file)
target_pred = restored_model.predict(features_test)
print('R2 score on individual targets',sklearn.metrics.r2_score(target_test, target_pred, multioutput='raw_values') )

R2 score on individual targets [0.66955375 0.6870105  0.5155454 ]
